In [1]:
import pandas as pd
from evaluate import load
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from seq2seq import create_transformers_train_data, train_transformer, decode_with_transformer

In [2]:
data = pd.read_csv('yelp_parallel/test_en_parallel.txt', sep='\t')
data.columns = ['Style1', 'Style2']
data = data[:5000]

In [3]:
sentences_negative = data['Style1'].values.tolist()
sentences_positive = data['Style2'].values.tolist()

In [4]:
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
instruction = 'Transform negative sentiment to positive sentiment: '
sentences_negative_with_ins = [f'{instruction}{s}' for s in sentences_negative]

In [6]:
train_set = create_transformers_train_data(sentences_negative_with_ins, sentences_positive, tokenizer)

C:\Users\vlade\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [8]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(model.parameters(), lr=0.001)
train_transformer(model, train_loader, optimizer, 3, device='cuda')

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/3, Loss: 2.0330
Epoch 2/3, Loss: 1.5512
Epoch 3/3, Loss: 1.2864


In [9]:
predicted_sentence = decode_with_transformer(sentences_negative_with_ins[0], tokenizer, model, device='cuda')
predicted_sentence

"ever since joes has changed hands it's been better and better."

In [10]:
reference_sentence = sentences_positive[0]
reference_sentence

"Ever since joes has changed hands it's gotten better and better."

In [11]:
bleu = load('bleu')
bleu.compute(predictions=[predicted_sentence], references=[[reference_sentence]])

{'bleu': 0.6340466277046861,
 'precisions': [0.8333333333333334,
  0.7272727272727273,
  0.6,
  0.4444444444444444],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 12,
 'reference_length': 12}

In [12]:
bertscore = load('bertscore')
bertscore.compute(predictions=[predicted_sentence], references=[reference_sentence], model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.9711467623710632],
 'recall': [0.9711467623710632],
 'f1': [0.9711467623710632],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.47.1)'}

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(model.parameters(), lr=0.0001)
train_transformer(model, train_loader, optimizer, 5, device='cuda')

Epoch 1/5, Loss: 2.3761
Epoch 2/5, Loss: 1.9710
Epoch 3/5, Loss: 1.8513
Epoch 4/5, Loss: 1.7691
Epoch 5/5, Loss: 1.7002


In [14]:
predicted_sentence = decode_with_transformer(sentences_negative_with_ins[0], tokenizer, model, device='cuda')
predicted_sentence

"since joes has changed hands it's just getting better and better."

In [15]:
bleu.compute(predictions=[predicted_sentence], references=[[reference_sentence]])

{'bleu': 0.6340466277046861,
 'precisions': [0.8333333333333334,
  0.7272727272727273,
  0.6,
  0.4444444444444444],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 12,
 'reference_length': 12}

In [16]:
bertscore.compute(predictions=[predicted_sentence], references=[reference_sentence], model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.9453003406524658],
 'recall': [0.9484442472457886],
 'f1': [0.9468696713447571],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.47.1)'}

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=data_collator)
optimizer = AdamW(model.parameters(), lr=0.0005)
train_transformer(model, train_loader, optimizer, 7, device='cuda')

Epoch 1/7, Loss: 2.0527
Epoch 2/7, Loss: 1.6677
Epoch 3/7, Loss: 1.4707
Epoch 4/7, Loss: 1.2858
Epoch 5/7, Loss: 1.1525
Epoch 6/7, Loss: 1.0059
Epoch 7/7, Loss: 0.8925


In [18]:
predicted_sentence = decode_with_transformer(sentences_negative_with_ins[0], tokenizer, model, device='cuda')
predicted_sentence

"ever since joes has changed hands it's gotten better and better."

In [19]:
bleu.compute(predictions=[predicted_sentence], references=[[reference_sentence]])

{'bleu': 0.9036020036098448,
 'precisions': [0.9166666666666666,
  0.9090909090909091,
  0.9,
  0.8888888888888888],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 12,
 'reference_length': 12}

In [20]:
bertscore.compute(predictions=[predicted_sentence], references=[reference_sentence], model_type='microsoft/deberta-xlarge-mnli')

{'precision': [0.9945324659347534],
 'recall': [0.9945324659347534],
 'f1': [0.9945324659347534],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.47.1)'}

In [43]:
# BLEU / T5: 0.63, 0.63, 0.90
# BARTScore / T5: 0.97, 0.95, 0.99   
# Со зголемување на епохите се добиваат подобри резултати
# Овие тестирања покажаа за нијанса послаби резултати од претходната задача
# Во seq2seq скриптата зголемен е опсегот од 10 на 128 зборови